In [ ]:
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

from IPython.display import display, clear_output
import glob
import logging
import numpy as np
import os
import scipy
import cv2

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import caiman as cm
from caiman.source_extraction import cnmf as cnmf
from caiman.paths import caiman_datadir
from caiman.utils.utils import download_demo
import matplotlib.pyplot as plt

import bokeh.plotting as bpl
try:
       from bokeh.io import vform, hplot
except:
       # newer version of bokeh does not use vform & hplot, instead uses column & row
       from bokeh.layouts import column as vform
       from bokeh.layouts import row as hplot
from bokeh.models import CustomJS, ColumnDataSource, Slider

bpl.output_notebook()

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm. Test the limits of your system before trying to process very large amounts of data.

In [ ]:
fnames = ['data_endoscope.tif']  # filename to be processed
fnames = [download_demo(fnames[0])]

In [ ]:
fr = 15                                                             # frame rate (Hz)
decay_time = 0.95                                                   # approximate length of transient event in seconds
gSig = (3, 3)                                                       # expected half size of neurons
gSiz = (13, 13)
ssub = 1
ds_factor = ssub
p = 0                                                               # order of AR indicator dynamics
min_SNR = 1.5                                                       # minimum SNR for accepting new components
min_pnr = 10
min_corr = 0.8
rval_thr = 0.85                                                     # correlation threshold for new component inclusion
mot_corr = True                                                     # flag for online motion correction 
pw_rigid = False                                                    # flag for pw-rigid motion correction (slower but potentially more accurate)
max_shifts_online = 10                                              # maximum allowed shift during motion correction
sniper_mode = False                                                 # flag using a CNN to detect new neurons (o/w space correlation is used)
init_batch = 300                                                    # number of frames for initialization (presumably from the first file)
expected_comps = 500                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
dist_shape_update = False                                           # flag for updating shapes in a distributed way
min_num_trial = 8                                                   # number of candidate components per frame     
K = None                                                            # initial number of components
epochs = 2                                                          # number of passes over the data
show_movie = False                                                  # show the movie with the results as the data gets processed

params_dict = {'fnames': fnames,
               'fr': fr,
               'decay_time': decay_time,
               'method_init': 'corr_pnr',
               'gSig': gSig,
               'gSiz': gSiz,
               'p': p,
               'epochs': 2,
               'ssub': ssub,
               'ds_factor': ssub,                                   # important to have ds_factor = ssub
               'min_SNR': min_SNR,
               'min_pnr': min_pnr,
               'min_corr': min_corr,
               'bas_nonneg': True,
               'center_psf': True,
               'rval_thr': rval_thr,
               'motion_correct': mot_corr,
               'init_batch': init_batch,
               'only_init': True,
               'init_method': 'cnmf',
               'normalize_init': False,
               'update_freq': 200,
               'expected_comps': expected_comps,
               'sniper_mode': sniper_mode,
               'dist_shape_update' : dist_shape_update,
               'min_num_trial': min_num_trial,
               'merge_thr': 0.65,
               'K': K,
               'epochs': epochs,
               'max_shifts_online': max_shifts_online,
               'pw_rigid': pw_rigid,
               'show_movie': show_movie}
opts = cnmf.params.CNMFParams(gnb=0, params_dict=params_dict)

In [ ]:
cnm = cnmf.online_cnmf.OnACID(params=opts)
cnm.fit_online()

In [ ]:
images = cm.load(fnames[0], subindices=slice(0,1000))
Cn, pnr = cm.summary_images.correlation_pnr(images[::1], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
cnm.estimates.nb_view_components(img=Cn, denoised_color='red');

In [ ]:
cnm.estimates.plot_contours_nb(img=Cn)

In [ ]:
cnm.estimates.plot_contours(img=Cn)

## Plot timing
The plot below shows the time spent on each part of the algorithm (motion correction, tracking of current components, detect new components, update shapes) for each frame. Note that if you displayed a movie while processing the data (`show_movie=True`) the time required to generate this movie will be included here.

In [ ]:
T_motion = 1e3*np.array(cnm.t_motion)
T_detect = 1e3*np.array(cnm.t_detect)
T_shapes = 1e3*np.array(cnm.t_shapes)
T_online = 1e3*np.array(cnm.t_online) - T_motion - T_detect - T_shapes
plt.figure()
plt.stackplot(np.arange(len(T_motion)), T_motion, T_online, T_detect, T_shapes)
plt.legend(labels=['motion', 'process', 'detect', 'shapes'], loc=2)
plt.title('Processing time allocation')
plt.xlabel('Frame #')
plt.ylabel('Processing time [ms]')
plt.ylim([0,140]);

In [ ]:
cnm.estimates.play_movie(images, magnification=3, q_max=99.5, q_min=0.5, include_bck=False)